# Improvement Carvana_local_learning_prod_ready

## Подключение библиотек и загрузка датасета

In [ ]:
# Улучшения: 
# Убрал .cuda() из класса модели и добавил в даталоадеры перенос на gpu
# Добавил трэйсинг моделии и ее сохранение, загрузку модели
# Заменил лишний ресайз масок на передачу оригинолов масок
# Сделал предсказания с разным размером батча
# Нейтрализовал проявления хардкода
# Написал комментарии и пояснения
# Сделал submission при обучении на всем трерировочном датасете

In [ ]:
# переписать pil на cv2 и сравнить производительность
# Реализовать аугментацию через albumintation
# Попробовать softdice loss + bce (как в dlcource.ai)
# Сделать нормализацию через albumintation (нужны скрипты для определения средего и дисперсии по каждому каналу)
# и сравнить точность (параметры можно взять из get_preprocessing_fn из smp если используем их модели)
# разобраться, заморожены ли веса энкодера у smp моделей
# реализовать модель из https://github.com/lyakaap/Kaggle-Carvana-3rd-Place-Solution/blob/master/model_pytorch.py

In [ ]:
!pip install segmentation_models_pytorch

In [15]:
import torch.nn as nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np
import glob
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
import time
from torch.autograd import Variable

In [ ]:
# Выполнять, если датасет не загружен
!pip install -q kaggle
!mkdir ~/.kaggle
!cp ~/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c carvana-image-masking-challenge
!unzip ~/carvana-image-masking-challenge.zip ~/carvana_dataset/

!unzip ~/carvana_dataset/train.zip -d ~/carvana_dataset/train
!unzip ~/carvana_dataset/test.zip -d ~/carvana_dataset/test
!unzip ~/carvana_dataset/train_masks.zip -d ~/carvana_dataset/train_masks

!unzip ~/carvana_dataset/train_hq.zip -d ~/carvana_dataset/train_hq
!unzip ~/carvana_dataset/test_hq.zip -d ~/carvana_dataset/test_hq

!unzip ~/carvana_dataset/train_masks.csv.zip  ~/carvana_dataset/
!unzip ~/carvana_dataset/sample_submission.csv.zip  ~/carvana_dataset/
!unzip ~/carvana_dataset/metadata.csv.zip  ~/carvana_dataset/

!rm ~/carvana-image-masking-challenge.zip
!rm ~/carvana_dataset/test.zip
!rm ~/carvana_dataset/train_masks.zip
!rm ~/carvana_dataset/train.zip
!rm ~/carvana_dataset/test_hq.zip
!rm ~/carvana_dataset/train_hq.zip
!rm ~/carvana_dataset/train_masks.csv.zip
!rm ~/carvana_dataset/sample_submission.csv.zip
!rm ~/carvana_dataset/metadata.csv.zip

In [16]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

## Используемые функции

In [17]:
def get_data_csv(imgs_path: str = None, masks_path: str = None) -> pd.DataFrame:
    '''Функция получает на вход пути к директориям с изображениями и масками
    и генерирует датафрейм, содержащий имя изображений, их адреса и адреса
    соответствующих им масок
  
    Входные параметры:
    imgs_path: str - путь к директории с изображениями,
    masks_path: str - путь к директории с масками
    Возвращаемые значения:
    pd.DataFrame: data - dataframe, содержащий адреса изображений и соответствующих им масок'''

    assert (imgs_path != None) & (masks_path != None)
    # imgs_path or masks_path is equal None

    data_img = {}
    data_mask = {}
    data_img['imgs_path'] = []
    data_mask['masks_path'] = []
    data_img['imgs_path'] = list(glob.glob(imgs_path + "/*"))
    data_mask['masks_path'] = list(glob.glob(masks_path + "/*"))

    data_img = pd.DataFrame(data_img)
    data_mask = pd.DataFrame(data_mask)

    def file_name(x):
        return x.split("/")[-1].split(".")[0]

    data_img["file_name"] = data_img["imgs_path"].apply(lambda x: file_name(x))
    data_mask["file_name"] = data_mask["masks_path"].apply(lambda x: file_name(x)[:-5])

    data = pd.merge(data_img, data_mask, on = "file_name", how = "inner")

    return data

In [18]:
def get_train_test(source_df: pd.DataFrame, separate_feature: str = None, test_size: float = 0.25) -> pd.DataFrame:
    '''Функция разделяет source_df на две части с коэффициентом test_size
    по уникальным значениям separate_feature так, чтобы в новых датафреймах
    не было строк с одинаковыми значенияти из separate_feature

    Входные параметры:
    source_df: pd.DataFrame - датафрейм для разделения на train и test
    separate_feature: str - поле, по которому датафрейм будет разделен
    test_size: float - коэффициент разделения дтафрейма
    Возвращаемые значения:
    pd.DataFrame: data_train - датафрейм для тренировки
    pd.DataFrame: data_valid - датафрейм для валидации'''
  
    if (separate_feature != None) & (separate_feature in source_df.columns):
        train_cars, valid_cars = train_test_split(source_df[separate_feature].unique(), test_size=test_size, random_state=42)
        data_valid = source_df[np.isin(source_df[separate_feature].values, valid_cars)]
        data_train = source_df[np.isin(source_df[separate_feature].values, train_cars)]
        assert source_df.shape[0] == (data_valid.shape[0] + data_train.shape[0])
        assert np.isin(data_train[separate_feature].values, data_valid[separate_feature].values).sum() == 0
    else:
        data_train, data_valid = train_test_split(source_df, test_size=test_size)

    return data_train, data_valid


In [19]:
def DICE(logits: torch.Tensor, targets: torch.Tensor, treashold: float) -> float:
    '''Функция для вычисления DICE коэффициента для набора изображенй в формате torch.Tensor
    Входные параметры:
    logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
    targets: torch.Tensor - тензор из целевых целевых значений масок
    treashold: float - порог для определения класса точки в предсказанной точке
    Возвращаемые значения:
    score: float - значение DICE коэффициента для набора предсказанных масок'''
    
    smooth = 1
    num = targets.size(0)
    probs = torch.sigmoid(logits)
    outputs = torch.where(probs > treashold, 1, 0)
    m1 = outputs.view(num, -1)
    m2 = targets.view(num, -1)
    intersection = (m1 * m2)

    score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
    score = score.sum() / num
    return score

In [20]:
def tensor_to_rle(tensor: torch.Tensor) -> str:
    '''Функция принимает одну маску в тензорном формате, элементы которой
    имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
    Входные параметры:
    tensor: torch.Tensor - маска в тензорном формате
    Возвращаемые значения:
    rle_str: str - rle представление маски в строком виде'''
    
    # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
    # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
    # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
    # не сильно упадет)
    tensor = tensor.view(1, -1)
    tensor = tensor.squeeze(0)
    tensor[0] = 0
    tensor[-1] = 0
    rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
    rle[1::2] = rle[1::2] - rle[:-1:2]
    rle = rle.cpu().detach().numpy()
    rle_str = rle_to_string(rle)
    return rle_str

In [21]:
def numpy_to_rle(mask_image: np.ndarray) -> str:
    '''Функция принимает одну маску в формате массива numpy, элементы которой
    имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
    Входные параметры:
    mask_image: numpy.ndarray - маска в тензорном формате
    Возвращаемые значения:
    rle_str: str - rle представление маски в строковом виде'''
    
    # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
    # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
    # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
    # не сильно упадет)
    pixels = mask_image.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle_str = rle_to_string(runs)
    return rle_str

In [22]:
def rle_to_string(runs: torch.Tensor) -> str:
    '''Функция преобразует последовательноть чисел в тензоре runs
    в строковое представление этой последовательности
    Входные параметры:
    runs: torch.Tensor - последовательность чисел в тензорном формате
    Возвращаемые значения:
    rle_str: str - строковое представление последовательности чисел'''
    
    return ' '.join(str(x) for x in runs)

In [23]:
def mask_to_rle(mask_addr: str) -> str:
    '''Функция преобразует маску, имеющую адрес mask_addr и сохраненную в
    формате .gif, элементы которой имеют значения 0 и 1 в rle представление
    в строковом виде
    Входные параметры:
    mask_addr: str - адрес маски
    Возвращаемые значения:
    mask_rle: str - rle представление маски в строком виде
    '''
    
    mask = Image.open(mask_addr).convert('LA') # преобразование в серый
    mask = np.asarray(mask).astype('float')[:,:,0]
    mask = mask/255.0
    mask_rle = numpy_to_rle(mask)
    return mask_rle

## Используемые классы

In [24]:
class DiceMetric(nn.Module):
    '''Класс для вычисления DICE коэффициента для набора изображенй в формате torch.Tensor
    с заданным порогом для определния класса каждой точки изображения'''
    
    def __init__(self, treashold: float=0.5):
        '''treashold: float - порог для определения класса точки в предсказанной точке'''
        super(DiceMetric, self).__init__()
        self.treashold = treashold

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> float:
        '''Входные параметры:
        logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
        targets: torch.Tensor - тензор из целевых целевых значений масок
        Возвращаемые значения:
        score: float - значение DICE коэффициента для набора предсказанных масок'''
        with torch.no_grad():
            smooth = 1
            num = targets.size(0)
            probs = torch.sigmoid(logits)
            outputs = torch.where(probs > self.treashold, 1., 0.)
            m1 = outputs.view(num, -1)
            m2 = targets.view(num, -1)
            intersection = (m1 * m2)

            score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
            score = score.sum() / num
            return score

In [25]:
class SoftDiceLoss(nn.Module):
    '''Класс для вычисления DICE loss для набора изображенй в формате torch.Tensor'''
    def __init__(self):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> float:
        '''Входные параметры:
        logits: torch.Tensor - тензор из предсказанных масок в logit масштабе
        targets: torch.Tensor - тензор из целевых целевых значений масок
        Возвращаемые значения:
        score: float - значение DICE loss для набора предсказанных масок'''
        smooth = 1
        num = targets.size(0)
        probs = torch.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score

In [26]:
class CustomDatasetForTrain(Dataset):
    '''Класс для создания тренировочных и валидационных датасетов'''
    def __init__(self, data_info: pd.DataFrame, device: str, out_shape: tuple=(512, 512), 
                                                             skip_mask: bool=False):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами изображений и масок
        device: str - имя устройства, на котором будут обрабатываться данные
        out_shape: tuple - пространственная размерность тензоров, к которой будут приводиться изображения и маски
        skip_mask: bool - флаг, нужно ли генерировать исходную маску (без изменения размерности)
        Возвращаемые значения:
        объект класса CustomDatasetForTrain'''
        # Подаем подготовленный датафрейм
        self.data_info = data_info
        # Разделяем датафрейм на rgb картинки 
        self.image_arr = self.data_info.iloc[:,0]
        # и на сегментированные картинки
        self.mask_arr = self.data_info.iloc[:,2]
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        # Устройство, на котором будут находиться выходные тензоры
        self.device = device
        # Пространственные размеры тензоров на выходе объекта
        self.out_shape = out_shape
        # Нужно ли пробрасывать маску изображения на выход без изменений
        self.skip_mask = skip_mask

    def __getitem__(self, index: int):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        img: torch.Tensor - тензорное представление изображения с размерностью out_shape
        mask_small: torch.Tensor - тензорное представление маски с исходной размерностью
        mask: torch.Tensor - тензорное представление изображения с размерностью out_shape 
        (возвращается если значение skip_mask равно True)'''
        img = np.asarray(Image.open(self.image_arr[index])).astype('float')
        img = (torch.as_tensor(img)/255.0).to(self.device)
        # unsqueeze - чтобы interpolate работало
        # permute - переставляем измерение каналов на 2-е место
        img = img.unsqueeze(0).permute(0,3,1,2)
        # clamp не позволяет выйти за границы значений
        img = F.interpolate(input=img, size=self.out_shape, align_corners=False, mode='bicubic').clamp(min=0, max=1)
        img = img.squeeze(0)
        # Маски - одноканальные изображения со значениями 0 и 1
        mask = Image.open(self.mask_arr[index])
        mask = np.asarray(mask).astype('float')
        # unsqueeze - добавляем измерение каналов
        mask = (torch.as_tensor(mask)).to(self.device).unsqueeze(0) 
        # unsqueeze - чтобы interpolate работало
        mask_small = mask.unsqueeze(0)
        mask_small = F.interpolate(input=mask_small, size=self.out_shape, mode='nearest')
        mask_small = mask_small.squeeze(0)
        
        # Если необходима исходная маска, то дополнительно возвращаем ее
        if self.skip_mask == True:
            return (img.float(), mask_small.float(), mask.float())
        else:
            return (img.float(), mask_small.float())

    def __len__(self):
        return self.data_len

In [27]:
class CustomDatasetForTest(Dataset):
    '''Класс для создания тренировочных и валидационных датасетов'''
    def __init__(self, data_info, device: str, out_shape: tuple=(512, 512)):
        '''Входные параметры:
        data_info: pd.DataFrame - датафрейм с адресами изображений
        device: str - имя устройства, на котором будут обрабатываться данные
        out_shape: tuple - пространственная размерность тензоров, к которой будут приводиться изображения
        Возвращаемые значения:
        объект класса CustomDatasetForTest'''
        # Подаем наш подготовленный датафрейм
        self.data_info = data_info
        # Получаем адреса RGB изображений 
        self.image_addresses = self.data_info.iloc[:,0]
        # Количество пар картинка-сегментация
        self.data_len = len(self.data_info.index)
        # Пространственные размеры тензоров на выходе объекта
        self.out_shape = out_shape
        # Устройство, на котором будут находиться выходные тензоры
        self.device = device

    def __getitem__(self, index):
        '''Входные параметры:
        img: int - индекс для обращения к элементам датафрейма data_info
        Возвращаемые значения:
        img: torch.Tensor - тензорное представление изображения с размерностью out_shape
        mask_small: torch.Tensor - тензорное представление маски с исходной размерностью
        image_name: str - имя изображения'''
        img = np.asarray(Image.open(self.image_addresses[index])).astype('float')
        img = (torch.as_tensor(img)/255).to(self.device)    
        # unsqueeze - чтобы interpolate работало
        # permute - переставляем измерение каналов на 2-е место
        img = img.unsqueeze(0).permute(0,3,1,2)
        # clamp не позволяет выйти за границы значений
        img = F.interpolate(input=img, size=self.out_shape, align_corners=False, mode='bicubic').clamp(min=0, max=1)
        img = img.squeeze(0)
        image_address = self.image_addresses[index]
        image_name = image_address.split('/')[-1]
    
        return (index, img.float(), image_name)

    def __len__(self):
        return self.data_len

In [28]:
class NeuralNetwork(nn.Module):
    '''Класс для создания работы с нейронной сетью для семантической сегментации Carvana'''
    def __init__(self, model: nn.Module):
        '''Конструктор класса
        Входные параметры:
        model: nn.Module - последовательность слоев или модель, через которую будут проходить данные
        Возвращаемые значения: 
        объект класса NeuralNetwork'''
        super(NeuralNetwork, self).__init__()
        self.model = model

    def forward(self, input_data):
        '''Функция прямого прохода через объкт класса
        Входные параметры:
        input_data: torch.Tensor - тензорное представление изображения
        Возвращаемые значения: 
        input_data: torch.Tensor - тензорное представление маски изображения'''
        output_data = self.model(input_data)
        return output_data
    
    @staticmethod
    def tensor_to_rle(tensor: torch.Tensor) -> str:
        '''Статический метод принимает одну маску в тензорном формате, элементы которой
        имеют значения 0. и 1. и генерирует rle представление маски в строковом формате
        Входные параметры:
        tensor: torch.Tensor - маска в тензорном формате
        Возвращаемые значения:
        rle_str: str - rle представление маски в строковом виде'''
    
        # Для правильной работы алгоритма необходимо, чтобы первое и последнее значения выпрямленной маски
        # (что соответствует двум углам изображения) были равны 0. Это не должно повлиять на качество работы
        # алгоритма, так как мы не ожидаем наличие объекта в этих точках (но даже если он там будет, качество
        # не сильно упадет)
        with torch.no_grad():
            tensor = tensor.view(1, -1)
            tensor = tensor.squeeze(0)
            tensor[0] = 0
            tensor[-1] = 0
            rle = torch.where(tensor[1:] != tensor[:-1])[0] + 2
            rle[1::2] = rle[1::2] - rle[:-1:2]
            rle = rle.cpu().detach().numpy()
            rle_str = NeuralNetwork.rle_to_string(rle)
            return rle_str
    
    @staticmethod
    def rle_to_string(runs: torch.Tensor) -> str:
        '''Функция преобразует последовательноть чисел в тензоре runs
        в строковое представление этой последовательности
        Входные параметры:
        runs: torch.Tensor - последовательность чисел в тензорном формате
        Возвращаемые значения:
        rle_str: str - строковое представление последовательности чисел'''
        return ' '.join(str(x) for x in runs)
    
    
    def fit(self, criterion: object, metric: object, optimizer: object, 
                  train_data_loader: DataLoader, valid_data_loader: DataLoader=None, epochs: int=1):
        '''Метод для обучения объекта класса
        Входные параметры:
        criterion: object - объект для вычисления loss
        metric: object - объект для вычисления метрики качества
        optimizer: object - оптимизатор
        train_data_loader: DataLoader - загрузчик данных для обучения
        valid_data_loader: DataLoader - загрузчик данных для валидации
        epochs: int - количество эпох обучения
        
        Возвращаемые значения:
        result: dict - словарь со значениями loss при тренировке, валидации и метрики при валидации 
        для каждой эпохи'''
        
        self.optimizer = optimizer
        epoch_train_losses = []
        epoch_valid_losses = []
        epoch_valid_metrics = []
        result = {}
        
        for epoch in range(epochs):
            self.model.train()
            time1 = time.time()
            running_loss =0.0
            train_losses = []
            for batch_idx, (data, labels) in enumerate(train_data_loader):
                data, labels = Variable(data), Variable(labels)        

                optimizer.zero_grad()
                outputs = self.model(data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                train_losses.append(loss.item())
                if (batch_idx+1) % 300 == 299:
                    print(f'Train Epoch: {epoch+1}, Loss: {running_loss/300}')
                    time2 = time.time()
                    print(f'Spend time for 300 batches: {time2-time1} sec')
                    time1 = time.time()
                    running_loss = 0.0

            train_loss = np.mean(train_losses)        
            
            
            if valid_data_loader != None:
                self.model.eval()
                valid_metrics = []
                valid_losses = []
                for batch_idx, (data, labels_small, labels) in enumerate(valid_data_loader):
                    data, labels, labels_small = Variable(data), Variable(labels), Variable(labels_small)
                    outputs = self.model(data)
                    # loss вычисляется для сжатых масок для правильной валидации (обучались на сжатых)
                    # чтобы вовремя определить переобучение
                    loss = criterion(outputs, labels_small)
                    valid_losses.append(loss.item())
                    #Преобразуем выход модели к размеру соответствующей маски
                    outputs = F.interpolate(input=outputs, size=(labels.shape[2], labels.shape[3]), mode='nearest')

                    # метрика считается для исходных размеров потому что именно так итоговое качество
                    # определяется алгоритмом kaggle 
                    metric_value = metric(outputs, labels)
                    valid_metrics.append(metric_value.item())
                    
                valid_loss    = np.mean(valid_losses)
                valid_metric  = np.mean(valid_metrics)
                print(f'Epoch {epoch+1}, train loss: {train_loss}, valid_loss: {valid_loss}, valid_metric: {valid_metric}')
            else:
                print(f'Epoch {epoch+1}, train loss: {train_loss}')
                valid_loss = None
                valid_metric = None
            
            epoch_train_losses.append(train_loss)
            epoch_valid_losses.append(valid_loss)
            epoch_valid_metrics.append(valid_metric)
        
        result['epoch_train_losses'] = epoch_train_losses
        result['epoch_valid_losses'] = epoch_valid_losses
        result['epoch_valid_metrics'] = epoch_valid_metrics
        
        return result
    
    
    
    def predict(self, test_data_loader: DataLoader, predict_directory: str, output_size: tuple=(1280, 1918), 
                mask_treashold: float=0.5, generate_rle_dataframe: bool=True) -> pd.DataFrame:
        '''Метод для предсказания масок для набора изображения
        Входные параметры:
        test_data_loader: DataLoader - загрузчик данных для предсказания
        predict_directory: str - директория, в которую будут сохраняться сгенерированные маски
        output_size: tuple - пространственная размерность выходных масок
        mask_treashold: float - порог, по которому будет определяться класс каждой точки для масок
        generate_rle_dataframe: bool - флаг, нужна ли генерация rle представлений масок
        Возвращаемые значения:
        rle_dataframe: pd.DataFrame - датафрейм с rle представлениями для масок (если 
        generate_rle_dataframe==True)
        Маски в формате .gif для изображений с соответствующими именами, находятся в директории predict_directory'''
        self.model.eval()
        img_names = []
        img_rles = []
        
        for batch_idx, (index, img, img_name)  in enumerate(test_data_loader):

            img = Variable(img)        
            pred_mask_logit = self.model(img)
            pred_mask_logit = F.interpolate(input=pred_mask_logit, size=output_size, mode='nearest')
            pred_mask_logit_prob = torch.sigmoid(pred_mask_logit)
            pred_mask = torch.where(pred_mask_logit_prob > mask_treashold, 1, 0)
            
            # Каждое изображение в тензоре преобразуем в картинку и сохраняем
            for i in range(pred_mask.shape[0]):
                mask = (pred_mask[i].cpu().numpy() * 255.0)[0] # [0] - избавляемся от батч размерности
                PIL_image = Image.fromarray(mask.astype('uint8'), 'L')
                PIL_image.save((predict_directory+img_name[i]).split('.')[0]+'.gif')
                
                # Если требуется, получаем значения rle для каждой картинки
                if generate_rle_dataframe == True:
                    img_names.append(img_name[i])
                    img_rles.append(NeuralNetwork.tensor_to_rle(pred_mask[i]))
                
        if generate_rle_dataframe == True:
            rle_dataframe = pd.DataFrame(list(zip(img_names, img_rles)), columns =['img_name', 'img_rle'])
            return rle_dataframe
    
    def save(self, path_to_save: str='./model.pth'):
        '''Метод сохранения весов модели
        Входные параметры:
        path_to_save: str - директория для сохранения состояния модели'''
        torch.save(self.model.state_dict(), path_to_save)
    
    def trace_save(self, path_to_save: str='./model.pth'):
        '''Метод сохранения модели через torchscript
        Входные параметры:
        path_to_save: str - директория для сохранения модели'''
        example_forward_input = torch.rand(1, 3, 512, 512).to('cpu')
        if next(self.model.parameters()).is_cuda:
            example_forward_input= example_forward_input.to('cuda:0')
            
        traced_model = torch.jit.trace(self.model, example_forward_input)
        torch.jit.save(traced_model, 'model.pt')
    
    def load(self, path_to_model: str='./model.pth'):
        '''Метод загрузки весов модели
        Входные параметры:
        path_to_model: str - директория с сохраненными весами модели'''
        self.model.load_state_dict(torch.load(path_to_model))

## Обучение модели

In [29]:
dataset_path = '/home/dima/carvana_dataset'
imgs_path  = dataset_path + '/train/train'
masks_path = dataset_path + '/train_masks/train_masks'

nn_image_shape = (512, 512)
learning_rate = 0.001
num_epochs = 5
mask_treashold = 0.5

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [30]:
data = get_data_csv(imgs_path=imgs_path, masks_path=masks_path)
    
# Добавляем признак, по которому будем разбивать датасет на train и test,
# чтобы не было разных фотографий одной и той же машины в двух датасетах
data["car"] = data["file_name"].apply(lambda x: x.split('_')[0])

In [31]:
# Обучение с валидацией
train_df, valid_df = get_train_test(data, separate_feature='car', test_size=0.25)
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)

train_data = CustomDatasetForTrain(train_df, device, out_shape=nn_image_shape)
valid_data = CustomDatasetForTrain(valid_df, device, out_shape=nn_image_shape, skip_mask=True)

train_data_loader = DataLoader(train_data,batch_size=4, shuffle=True)
valid_data_loader = DataLoader(valid_data,batch_size=4, shuffle=False)

In [32]:
# Обучение без валидации
train_data = CustomDatasetForTrain(data, device, out_shape=nn_image_shape)
train_data_loader = DataLoader(train_data,batch_size=4, shuffle=True)

In [33]:
# Создаем модель на основе предложенной архитектуры
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16]).to(device)

#model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
#                 encoder_weights='imagenet').to(device)

my_model = NeuralNetwork(model=model)

In [35]:
criterion = SoftDiceLoss()
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate)
metric = DiceMetric(treashold=mask_treashold)

In [36]:
my_model.fit(criterion,
             metric,
             optimizer,
             train_data_loader,
             valid_data_loader,
             epochs=num_epochs)

Train Epoch: 1, Loss: 0.0499671345949173
Spend time for 300 batches: 103.78062438964844 sec


KeyboardInterrupt: 

In [395]:
# Сохраняем веса обученной модели
my_model.save(path_to_save = './model.pth')

In [396]:
# Сохраняем оттрассированную модель
my_model.trace_save(path_to_save = './model.pt')

## Загрузка сохраненной модели

In [16]:
# Воспроизводим модель по известной архитектуре и сохраненным весам
model = smp.Unet('mobilenet_v2', classes=1, encoder_depth=5, 
                 encoder_weights='imagenet', decoder_channels = [256, 128, 64, 32, 16]).to(device)

my_model = NeuralNetwork(model=model)
my_model.load(path_to_model = './model.pth')

In [448]:
# Загружаем оттрассированную модель
my_model = torch.jit.load('./model.pt')
my_model = NeuralNetwork(model=my_model)
my_model = my_model.to(device)

## Предсказание модели

In [46]:
predict_directory = '/home/dima/carvana_dataset/test/predict_small/'
test_dataset = '/home/dima/carvana_dataset/test/test/'

In [47]:
test_dataframe = {}
test_dataframe['img_addr'] = list(glob.glob(test_dataset + "/*"))
test_dataframe = pd.DataFrame(test_dataframe)

In [48]:
test_data = CustomDatasetForTest(test_dataframe)
test_data_loader = DataLoader(test_data, batch_size=2, shuffle=False)

In [ ]:
rle_dataframe = my_model.predict(test_data_loader, predict_directory, 
                                 mask_treashold=mask_treashold, generate_rle_dataframe=True)

In [21]:
# Получаем датафрейм с результатом для заливки на kaggle
rle_dataframe.to_csv('rle_dataframe.csv', index=True)
sample_submission = pd.read_csv('/home/dima/carvana_dataset/sample_submission.csv')
sample_submission = sample_submission.merge(rle_dataframe, how='left', left_on='img', right_on='img_name')
sample_submission.drop(columns=['rle_mask', 'img_name'], inplace=True)
sample_submission.rename(columns={'img_rle': 'rle_mask'}, inplace=True)
sample_submission.to_csv('submission_02_10.csv', index=False)

In [28]:
torch.cuda.empty_cache()